In [ ]:
!pip install pyomo==6.7.1

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [ ]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [ ]:
schedule.Prestation.unique()

In [ ]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [ ]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [ ]:
sched_one_day.Prestation.value_counts()

In [ ]:
sched_one_day = sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [ ]:
sched_one_day["idx"] = sched_one_day.index

In [ ]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [ ]:
sched_one_day

In [ ]:
caregivers

In [ ]:
schedule

In [ ]:
# Import pyomo libraries
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product

In [ ]:
test = sched_one_day["Heure de début"].iloc[0] - pd.to_datetime(
    "2024-01-03 00:00:00"
)

test.seconds // 60

In [ ]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            clients = pd.read_excel(excel_file, sheet_name=1)
            sessions = pd.read_excel(excel_file, sheet_name=0)

            # NOTE: remove later
            sessions = sessions.loc[sessions.Date == "2024-01-03", :]
            sessions["idx"] = sessions.index

            # create duration
            common_date = sessions["Date"].iloc[0]
            sessions["Heure de fin"] = sessions["Heure de fin"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Heure de début"] = sessions["Heure de début"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Duration"] = (
                sessions["Heure de fin"] - sessions["Heure de début"]
            )
            sessions["Duration"] = sessions["Duration"].apply(
                lambda x: x.seconds // 60
            )

            sessions["Start_time"] = sessions[
                "Heure de début"
            ] - pd.to_datetime("2024-01-03 00:00:00")
            sessions["Start_time"] = sessions["Start_time"].apply(
                lambda x: x.seconds // 60
            )

            # discard
            discard_list = [
                "ADMINISTRATION",
                "VISITE MEDICALE",
                "FORMATION",
                "COORDINATION",
                "HOMMES TOUTES MAINS",
            ]
            sessions = sessions[~sessions.Prestation.isin(discard_list)]

            sessions = sessions.merge(clients, how="left", on="ID Client")
            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(excel_file, sheet_name=2)
            # TODO: take kaans matrix -> filter cargeivers if they are not available today
        except FileNotFoundError:
            print("Caregiver data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        """
        Generate mapping of cases IDs to case time for the procedure
        Returns:
            (dict): dictionary with CaseID as key and median case time (mins) for procedure as value
        """
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_start_time(self, tasks):
        tasks_df = pd.DataFrame(tasks, columns=["idx", "Cargegiver_ID"])

        temp = pd.DataFrame(
            self.df_sessions["Start_time"].values,
            index=self.df_sessions["idx"],
            columns=["Start_time"],
        )  # .to_dict()

        temp = tasks_df.merge(
            temp, how="left", right_index=True, left_on="idx"
        )

        arrays = [
            temp["idx"].to_list(),
            temp["Cargegiver_ID"].to_list(),
        ]
        tuples = list(zip(*arrays))

        index = pd.MultiIndex.from_tuples(
            tuples, names=["idx", "Cargegiver_ID"]
        )
        temp = pd.Series(temp["Start_time"].values, index=index)

        return temp.to_dict()

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, cargiver) in product(cases, cases, cargivers):
            if (case1 != case2) and (
                case2,
                case1,
                cargiver,
            ) not in disjunctions:
                disjunctions.append((case1, case2, cargiver))

        return disjunctions

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(
            initialize=model.CASES * model.CAREGIVERS, dimen=2
        )
        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )
        # Start time of a case
        model.CASE_START_TIME = pe.Param(
            model.TASKS, initialize=self._generate_start_time(model.TASKS)
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M
        # max_util = 0.85

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.TASKS, domain=pe.Binary)
        # Downtime of a caregiver caregiver
        model.DOWN_TIME = pe.Var(
            model.CAREGIVERS, bounds=(0, ub), within=pe.PositiveIntegers
        )
        # Session utilisation
        # model.UTILISATION = pe.Var(model.CAREGIVERS, bounds=(0, max_util), within=pe.PositiveReals)

        model.DISJUNCTIONS = pe.Set(
            initialize=self._generate_disjunctions(), dimen=3
        )

        # Objective
        def objective_function(model):
            return pe.summation(model.DOWN_TIME)
            # return sum([model.SESSION_ASSIGNED[case, session] for case in model.CASES for session in model.CAREGIVERS])
            # return min(model.UTILISATION.extract_values().values())

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # Constraints
        # TODO: get minimum working hours
        # Case start time must be after 8 AM
        # def case_start_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] >= self.df_sessions["Heure de début"].min()
        # model.CASE_START = pe.Constraint(model.TASKS, rule=case_start_time)

        # TODO: create maximum working hours
        # # Case end time must be before general working hours 22 PM
        # def case_end_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] + model.CASE_DURATION[case] <= self.df_sessions["Heure de fin"].max()
        # model.CASE_END_TIME = pe.Constraint(model.TASKS, rule=case_end_time)

        # Cases can be assigned to a maximum of one caregiver
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, session)]
                        for session in model.CAREGIVERS
                    ]
                )
                <= 1
            )

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )

        # Constraint 6: Utilisation is defined as fraction of a theatre session taken up by cases
        def theatre_util(model, session):
            return model.DOWN_TIME[session] == 1440 - sum(
                [
                    model.SESSION_ASSIGNED[case, session]
                    * model.CASE_DURATION[case]
                    for case in model.CASES
                ]
            )

        model.THEATRE_UTIL = pe.Constraint(model.CAREGIVERS, rule=theatre_util)

        # Constraint 6: Make sure the caregivers have shifts of 9 hours
        # somehow this does not work because of non constant pyomo expressions
        # def working_hours(model, caregiver):
        #     return 540 >= max([model.SESSION_ASSIGNED[case, caregiver]*model.CASE_START_TIME[case, caregiver] for case in model.CASES]) - min([model.SESSION_ASSIGNED[case, caregiver]*model.CASE_START_TIME[case, caregiver] for case in model.CASES])
        # model.WORKING_HOURS = pe.Constraint(model.CAREGIVERS, rule=working_hours)

        # TODO: -> question 3 if someone has no work
        # def compare_utils(model, caregiver):
        #     return model.UTILISATION[caregiver] >= 0.13
        # model.ALL_MIN = pe.Constraint(model.CAREGIVERS, rule=compare_utils)

        # TODO: create bounding box deadlines
        # Constraint 4: Cases must be completed before their target deadline
        # def set_deadline_condition(model, case, session):
        #     return model.SESSION_DATES[session] <= model.CASE_DEADLINES[case] + ((1 - model.SESSION_ASSIGNED[case, session])*model.M)
        # model.APPLY_DEADLINE = pe.Constraint(model.TASKS, rule=set_deadline_condition)

        # Constraint 5: No two cases can overlap
        def no_case_overlap(model, case1, case2, caregiver):
            return [
                model.CASE_START_TIME[case1, caregiver]
                + model.CASE_DURATION[case1]
                <= model.CASE_START_TIME[case2, caregiver]
                + (
                    (
                        2
                        - model.SESSION_ASSIGNED[case1, caregiver]
                        - model.SESSION_ASSIGNED[case2, caregiver]
                    )
                    * model.M
                ),
                model.CASE_START_TIME[case2, caregiver]
                + model.CASE_DURATION[case2]
                <= model.CASE_START_TIME[case1, caregiver]
                + (
                    (
                        2
                        - model.SESSION_ASSIGNED[case1, caregiver]
                        - model.SESSION_ASSIGNED[case2, caregiver]
                    )
                    * model.M
                ),
            ]

        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(
            model.DISJUNCTIONS, rule=no_case_overlap
        )

        pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        # Create solver (synchronous solve)
        solver = pe.SolverFactory("cbc", executable="/opt/homebrew/bin/cbc")

        # Add solver parameters (time limit)
        options = {"seconds": 60}
        for key, value in options.items():
            solver.options[key] = value

        # Solve model (verbose)
        solver_results = solver.solve(self.model, tee=True)
        return solver_results

In [ ]:
# min(scheduler.model.UTILISATION.extract_values().values()) # .display()

In [ ]:
scheduler = CareScheduler(excel_file)

In [ ]:
solver_results = scheduler.solve()

In [ ]:
model = scheduler.model

In [ ]:
model.DOWN_TIME.display()  # .extract_values()

In [ ]:
actions = [
    k for k, v in model.SESSION_ASSIGNED.extract_values().items() if v == 1
]

In [ ]:
actions[0]

In [ ]:
actions_df = pd.DataFrame(actions, columns=["idx", "Caregiver_ID"])

In [ ]:
temp = scheduler.df_sessions.copy()
temp = temp.merge(actions_df, how="left", on="idx")
temp

In [ ]:
temp.Caregiver_ID.isna().mean()

In [ ]:
temp.style

In [ ]:
# model.pprint()
# model.x.pprint()

In [ ]:
import plotly.express as px

In [ ]:
def plot_agenda_for_intervenant(intervenant_id):
    intervenant_agenda = temp[temp["Caregiver_ID"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Heure de début"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["Heure de fin"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["Caregiver_ID"].astype(str)

    fig = px.timeline(
        df_timeline,
        x_start="Start",
        x_end="Finish",
        y="Task",
        color="Resource",
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")

    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

In [ ]:
for intervenant_id in temp["Cargegiver_ID"].unique():
    plot_agenda_for_intervenant(intervenant_id)